In [1]:
import numpy as np
from torchvision.datasets import MNIST

Nesse notebook, será construído uma rede neural à mão que consegue reconhecer as imagens dos números do conjunto de dados [MNIST](https://en.wikipedia.org/wiki/MNIST_database). Foram utilizadas as mesmas classes e funções do notebook [XOR à mão](xor_from_scratch.ipynb).

In [3]:
class Sigmoid:
    def f(self, x):
        return 1 / (1 + np.exp(-x))
    def f_prime(self, x):
        return self.f(x) * (1-self.f(x))

In [4]:
class MSELoss:
    def f(self, y_hat, y):
        return np.sum((y_hat - y)**2)/len(y_hat)
    def f_prime(self, y_hat, y):
        return 2*(y_hat - y)

In [5]:
class Layer:
    def __init__(self, n_of_inputs: int, n_of_neurons: int , activation, bias: float=0.0):
        self.n_of_inputs = n_of_inputs
        self.n_of_neurons = n_of_neurons
        self.activation = activation
        self.bias = np.ones((1, n_of_neurons)) * bias 
        self.weights = np.random.uniform(-1, 1, (n_of_inputs, n_of_neurons)) 
        
        # As variáveis abaixo são necessárias para o backward
        self.weight_gradient = None  
        self.bias_gradient = None 
        self.layer_inputs = None # output da camada anterior, ou as entradas da rede caso for a primeira camada
        self.linear_output = None # resultado antes de ser aplicada a função de ativação -> linear_output = a @ w + b

    def forward(self, x):
        """
        Forward propagation da camada
        """
        self.layer_inputs = x 
        dot_product = self.layer_inputs @ self.weights 
        self.linear_output = dot_product + self.bias
        output = self.activation.f(self.linear_output)
        return output

    def backward(self, chain_rule_derivatives):
        """
        Cálculo dos gradientes da camada. 
        É calculada as derivadas em relação a matriz de pesos e o bias da camada (dC_dw e dC_db), e a 
        derivada em relação ao linear_output (dC_da), para que possa mandar essa derivada para trás para calcular
        o gradiente dos pesos das camadas anteriores, conforme o diagrama
        Parâmetros:
        chain_rule_derivatives - derivada calculada através da regra da cadeia, que foi mandada da camada seguinte (dC_da1)
        Retorno:
        updated_chain_rule_derivatives - derivada calculada através da regra da cadeia, para ser mandada para a camada anterior (dc_da0)
        """
        da1_dz = self.activation.f_prime(self.linear_output) 
        dz_dw = self.layer_inputs
        dz_da0 = self.weights
        
        dC_dw = dz_dw.T @ (da1_dz * chain_rule_derivatives) 
        dC_db = 1 * da1_dz * chain_rule_derivatives
        dC_da0 = (chain_rule_derivatives * da1_dz) @ dz_da0.T
        
        updated_chain_rule_derivatives = dC_da0
        self.weight_gradient = dC_dw
        self.bias_gradient = dC_db
        
        return updated_chain_rule_derivatives

In [6]:
class NeuralNetwork:
    def __init__(self, input_size, lr):
        self.layers = []
        self.input_size = input_size
        self.lr = lr

    def forward(self, x):
        """
        Forward propagation da rede
        """
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, loss_derivative):
        """
        Backward propagation da rede.
        Calcula os gradientes e aplica o algoritmo de gradiente descendente para atualizar os pesos e os bias
        """
        # Cálculo dos gradientes
        chain_rule_derivatives = loss_derivative
        for layer in reversed(self.layers):
            chain_rule_derivatives = layer.backward(chain_rule_derivatives)
        
        # Gradiente descendente
        for layer in self.layers:
            layer.weights -= layer.weight_gradient * self.lr
            layer.bias -= layer.bias_gradient * self.lr

    # Faz o forward chamando o objeto, passando os inputs como parâmetro, da mesma forma que o PyTorch faz
    def __call__(self, inputs):
        return self.forward(inputs)

    def append_layer(self, output_number: int, activation, bias: float=0.0):
        """
        Dado um número de saída adiciona uma camada ao fim da rede neural
        Ex: nn = NeuralNetwork(...)
          nn.append_layer(...)
          nn.append_layer(...)
          ...
        """
        # Caso seja a primeira camada
        if len(self.layers) == 0:
            new_layer_input = self.input_size
        else:
            new_layer_input = self.layers[-1].n_of_neurons

        self.layers.append(Layer(new_layer_input, output_number, activation, bias))

# Dataset MNIST
Esse dataset é dividido em dois conjuntos nomeados **train** e **test**, em que o primeiro possui 60000 imagens e o segundo 10000.<br>
Cada conjunto é dividido em **data**, que contêm as matrizes de pixeis da imagem, e **targets**, que contêm os números respectivos às matrizes de pixeis.
Visualizações de como a rede neural funciona para esse problema podem ser encontradas no [MNIST PyTorch](mnist_pytorch.ipynb)
## One hot encoding
A saída da rede neural está na forma *one hot*, ou seja, um vetor com 10 posições, em que cada index é respectivo ao número da probabilidade gerada. O dataset MNIST vem com os targets em forma de número, sendo assim, nessa implementação é necessário converte-los para one hot.
### Exemplos:
**9** -> [0 0 0 0 0 0 0 0 0 1]<br>
**4** -> [0 0 0 0 1 0 0 0 0 0]<br>
**7** -> [0 0 0 0 0 0 0 1 0 0]<br>

In [7]:
def one_hot(value: int):
    one_hot_vec = np.zeros((1, 10))
    one_hot_vec[0][value] = 1
    return one_hot_vec

In [8]:
mnist_train = MNIST(root='./data', train=True, download=False)
mnist_test = MNIST(root='./data', train=False, download=False) 

train_data = np.array(mnist_train.data)
train_targets = np.array([one_hot(t.item()) for t in mnist_train.targets])

test_data = np.array(mnist_test.data)
test_targets = np.array([one_hot(t.item()) for t in mnist_test.targets])

C:\Users\Vitor\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
lr = 0.003
criterion = MSELoss()
model = NeuralNetwork(28*28, lr)
model.append_layer(64, activation=Sigmoid(), bias=1)
model.append_layer(64, activation=Sigmoid(), bias=1)
model.append_layer(10, activation=Sigmoid(), bias=1)

for epoch in range(10):
    total_loss = 0
    for i, training_sample in enumerate(zip(train_data, train_targets)):
        x = training_sample[0].reshape(1,-1)
        y = training_sample[1]
        y_hat = model(x)
        loss = criterion.f(y_hat, y)
        loss_derivative = criterion.f_prime(y_hat, y)
        model.backward(loss_derivative)
        total_loss += loss
    print(f"Loss: {total_loss / len(train_data)} - Epoch: {epoch + 1}")

    # Validar
    hits = 0
    for x, y in zip(test_data, test_targets):
        y_hat = model(x.reshape(1, -1))
        if np.argmax(y_hat) == np.argmax(y):
            hits += 1
    print(f'\nEpoch Accuracy: {hits/len(test_data)* 100}%\n')

<ipython-input-3-53f0d3a21cd6>:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Loss: 0.6638958228340215 - Epoch: 1

Epoch Accuracy: 67.71000000000001%

Loss: 0.43152392365675235 - Epoch: 2

Epoch Accuracy: 76.75%

Loss: 0.36566674765549695 - Epoch: 3

Epoch Accuracy: 78.97999999999999%

Loss: 0.3224660885218194 - Epoch: 4

Epoch Accuracy: 82.28999999999999%

Loss: 0.2954003052815908 - Epoch: 5

Epoch Accuracy: 82.37%

Loss: 0.2924789001781059 - Epoch: 6

Epoch Accuracy: 83.03%

Loss: 0.26539080887528044 - Epoch: 7

Epoch Accuracy: 85.22%



Nota-se que o modelo obteve uma alta acurácia, considerando que treinou apenas por 10 épocas, utilizando uma função custo e um otimizador simples (MSELoss e Gradiente Descendente). No [MNIST PyTorch](mnist_pytorch.ipynb), o modelo treinado fica com uma acurácia melhor, devido ao leque de possibilidades que o PyTorch oferece para montar e treinar o modelo.

In [ ]:
def guess():
    # Gera um número aleatório no conjunto de dados de teste e o modelo tenta prever qual é esse número 
    n = np.random.randint(0, len(test_data))
    predicted = model.forward(test_data[n].reshape(1, -1))
    actual = test_targets[n]
    print(f"Actual number: {np.argmax(actual)} - Predicted number: {np.argmax(predicted)}")

In [ ]:
guess()